<center><h2>Regression<h2></center>

## Get Started

- Make sure opalite is running on Chrome or FireFox.
- Click *Cell* and *Run All* to kick it off (find it in the menu on top of this page) 
    - See the little circle on the upper right corner. It will be filled when the kernel is running and hollow when idle
    - If see the warnings of "Widget Javascript not detected.", wait a while and then click *Cell* and *Run All* again.
- Click *show/hide the raw code* button to show or hide the code
- Click *Insert* and *Insert Cell Above/Below* to insert a code block and start to write your own customized code (need to show code first)

In [1]:
# toggle code

from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show/hide the raw code."></form>''')

In [2]:
# load packages and check package versions

import sys
import os
import re
import datetime
import getpass
import logging
import sklearn
from sklearn.base import clone
#import forest_ci as fci
#sys.path.append('C:\\Users\\bihan\\eclipse\\workspace-python\\mltemplate')

os.environ['MLTEMPLATE_LOG_PATH'] = os.environ['HOME']

%matplotlib inline
%config IPCompleter.greedy=True
%autosave 0
# %load_ext autoreload
# %autoreload 2

import pandas as pd
import numpy as np
import plotly.graph_objs as go
from mltemplate.mldata import MLData
from mltemplate.mlmodel import MLModel
from ipywidgets import *
from IPython.display import display, clear_output, Image, Javascript, SVG
from plotly.offline import init_notebook_mode, iplot
from plotly.tools import FigureFactory as FF       

# Initialize data and model object instances
mld = MLData(regret=True)
mlm = MLModel()

# for plotly
init_notebook_mode(connected=True)

Autosave disabled


In [3]:
# from blkbms.bms import BMS
# from blkbms.message import Message, SemaphoreMsgBody
# from blkcore.dates.utils import now
# import getpass
# user = getpass.getuser()
# #print('Sending gen_stat')
# testmap = {'client': 'ADS',
#           'module': 'ADS-OPAL',
#           'submodule': 'Opal-regression',
#           'tag_def': 'version-1.0',
#           'key_1': user}
# with BMS() as conn:    
#     bms_message = Message()
#     bms_message.init(SemaphoreMsgBody(testmap), 103)
#     bms_message.get_header().set_msg_subtype(7)
#     try:        
#         response = conn.send(bms_message, timeout=100)
#     except:
#         pass
# #print('Done Sending gen_stat : ' + str(testmap))

In [4]:
# from blkbms.bms import BMS
# from blkbms.message import Message, SemaphoreMsgBody
# from blkcore.dates.utils import now
# import getpass
# user = getpass.getuser()

# def perf_log(client='ADS-OPAL', module='Opal-regression', submodule='start', tag_def='version-1.0', user=user):
#     testmap = {'client': client,
#               'module': module,
#               'submodule': submodule,
#               'tag_def': tag_def,
#               'key_1': user}
#     with BMS() as conn:    
#         bms_message = Message()
#         bms_message.init(SemaphoreMsgBody(testmap), 103)
#         bms_message.get_header().set_msg_subtype(7)
#         try:        
#             response = conn.send(bms_message, timeout=100)
#         except:
#             pass    

ModuleNotFoundError: No module named 'blkbms'

In [ ]:
# perf_log()

In [ ]:
import functools
import scipy.interpolate
import itertools
import math
#import numpy as np
from scipy.optimize import minimize
from scipy.signal import fftconvolve
from scipy.stats import norm
import copy
from sklearn.ensemble.forest import _generate_sample_indices

__all__ = ["gfit", "gbayes","calibrateEB", "calc_inbag", "random_forest_error", "_bias_correction",
           "_core_computation"]

'''
from .due import _due, _BibTeX
_due.cite(_BibTeX("""
@ARTICLE{Wager2014-wn,
  title       = "Two modeling strategies for empirical Bayes estimation.",
  author      = Efron, Bradley
  journal     = "Stat. Sci.",
  volume      =  29,
  number      =  2,
  pages       = "285--301",
  month       =  feb,
  year        =  2014,}"""),
          description=("Confidence Intervals for Random Forests:",
                       "The Jackknife and the Infinitesimal Jackknife"),
          path='forestci')
'''

def gfit(X, sigma, p=5, nbin=1000, unif_fraction=0.1):
    """
    Fit an empirical Bayes prior in the hierarchical model
        mu ~ G, X ~ N(mu, sigma^2)
    Parameters
    ----------
    X: ndarray
        A 1D array of observations
    sigma: float
        noise estimate on X
    p: int
        tuning parameter -- number of parameters used to fit G
    nbin: int
        tuning parameter -- number of bins used for discrete approximation
    unif_fraction: float
        tuning parameter -- fraction of G modeled as "slab"
    Returns
    -------
    An array of the posterior density estimate g
    Notes
    -----
    .. [Efron2014] B Efron. "Two modeling strategies for empirical Bayes
        estimation." Stat. Sci., 29(2): 285–301, 2014.
    """
    #global xvals, XX

    min_x = min(min(X) - 2 * np.std(X,ddof=1), 0)
    max_x = max(max(X) + 2 * np.std(X,ddof=1), np.std(X,ddof=1))
    xvals = np.linspace(min_x, max_x, nbin)
    binw = (max_x - min_x) / (nbin - 1)
    
    zero_idx = max(np.where(xvals <= 0)[0])
    noise_kernel = norm().pdf(xvals / sigma) * binw / sigma

    if zero_idx > 0:
        noise_rotate = noise_kernel[list(np.arange(zero_idx, len(xvals))) +
                                    list(np.arange(0, zero_idx))]
    else:
        noise_rotate = noise_kernel

    XX = np.zeros((p, len(xvals)), dtype=np.float)
    for ind, exp in enumerate(range(1,p+1)):
        mask = np.ones_like(xvals)
        mask[np.where(xvals <= 0)[0]] = 0
        XX[ind, :] = pow(xvals, exp) * mask
    XX = XX.T

    def neg_loglik(eta):
        mask = np.ones_like(xvals)
        mask[np.where(xvals <= 0)[0]] = 0
        g_eta_raw = np.exp(np.dot(XX, eta)) * mask
        if ((sum(g_eta_raw) == math.inf) |(sum(g_eta_raw) <= 100 * np.finfo(np.double).tiny)):
            return (1000 * (len(X) + sum(eta ** 2)))

        g_eta_main = g_eta_raw / sum(g_eta_raw)
        g_eta = (1 - unif_fraction) * g_eta_main + unif_fraction * mask / sum(mask)
        f_eta = fftconvolve(g_eta, noise_rotate, mode='same') 
        return np.sum(np.interp(X, xvals, -np.log(np.maximum(f_eta, 0.0000001))))


    eta_hat = minimize(neg_loglik, list(itertools.repeat(-1, p))).x  ###
    g_eta_raw = np.exp(np.dot(XX, eta_hat)) * mask
    g_eta_main = g_eta_raw / sum(g_eta_raw)
    g_eta = ((1 - unif_fraction) * g_eta_main + unif_fraction * mask) / sum(mask)

    return xvals, g_eta

def gbayes(x0, g_est, sigma):
    """
    Bayes posterior estimation with Gaussian noise
    Parameters
    ----------
    x0: ndarray
        an observation
    g_est: float
        a prior density, as returned by gfit
    sigma: int
        noise estimate
    Returns
    -------
    An array of the posterior estimate E[mu | x0]
    Notes
    -----
     .. [Efron2014] B Efron. "Two modeling strategies for empirical Bayes
          estimation." Stat. Sci., 29(2): 285–301, 2014.
    """

    Kx = norm().pdf((g_est[0] - x0) / sigma)
    post = Kx * g_est[1]
    post = post/ sum(post)
    return sum(post * g_est[0])

def calibrateEB(variances, sigma2, speedup=False):
    """
    Empirical Bayes calibration of noisy variance estimates
    Parameters
    ----------
    vars: ndarray
        list of variance estimates
    sigma2: int
        estimate of the Monte Carlo noise in vars
    speedup: boolean
        whether to use interpolation to speed up computations
    Returns
    -------
    An array of the calibrated variance estimates
    """

    if (sigma2 <= 0 or min(variances) == max(variances)):
        return(np.maximum(variances, 0))
    
    sigma = sigma2**0.5
    eb_prior = gfit(variances, sigma)
    
    if speedup:
        # If there are many points use interpolation to speed up computations
        calib_x = np.percentile(variances, np.arange(0, 100.1, .1))
        #calib_y = list(map(gbayes(xx, eb_prior, sigma), calib_x))
        calib_y = list(map(functools.partial(gbayes, g_est=eb_prior, sigma=sigma),
                      calib_x))
        calib_all = np.interp(variances, calib_x, calib_y)
    else: 
        calib_all = list(map(functools.partial(gbayes, g_est=eb_prior, sigma=sigma),
                        variances))
        
    return(calib_all)


def calc_inbag(n_samples, forest):
    """
    Derive samples used to create trees in scikit-learn RandomForest objects.

    Recovers the samples in each tree from the random state of that tree using
    :func:`forest._generate_sample_indices`.

    Parameters
    ----------
    n_samples : int
        The number of samples used to fit the scikit-learn RandomForest object.

    forest : RandomForest
        Regressor or Classifier object that is already fit by scikit-learn.

    Returns
    -------
    Array that records how many times a data point was placed in a tree.
    Columns are individual trees. Rows are the number of times a sample was
    used in a tree.
    """
    if not forest.bootstrap:
        e_s = "Cannot calculate the inbag from a forest that has "
        e_s = " bootstrap=False"
        raise ValueError(e_s)

    n_trees = forest.n_estimators
    inbag = np.zeros((n_samples, n_trees))
    sample_idx = []
    for t_idx in range(n_trees):
        sample_idx.append(
            _generate_sample_indices(forest.estimators_[t_idx].random_state,
                                     n_samples))
        inbag[:, t_idx] = np.bincount(sample_idx[-1], minlength=n_samples)
    return inbag


def _core_computation(X_train, X_test, inbag, pred_centered, n_trees):
    cov_hat = np.zeros((X_train.shape[0], X_test.shape[0]))

    for t_idx in range(n_trees):
        inbag_r = (inbag[:, t_idx] - 1).reshape(-1, 1)
        pred_c_r = pred_centered.T[t_idx].reshape(1, -1)
        cov_hat += np.dot(inbag_r, pred_c_r) / n_trees
    V_IJ = np.sum(cov_hat ** 2, 0)
    return V_IJ


def _bias_correction(V_IJ, inbag, pred_centered, n_trees):
    n_train_samples = inbag.shape[0]
    n_var = np.mean(np.square(inbag[0:n_trees]).mean(axis=1).T.view() -
                    np.square(inbag[0:n_trees].mean(axis=1)).T.view())
    boot_var = np.square(pred_centered).sum(axis=1) / n_trees
    bias_correction = n_train_samples * n_var * boot_var / n_trees
    V_IJ_unbiased = V_IJ - bias_correction
    return V_IJ_unbiased


def random_forest_error(forest, X_train, X_test, inbag=None, calibrate = True, speedup=False):
    """
    Calculates error bars from scikit-learn RandomForest estimators.

    RandomForest is a regressor or classifier object
    this variance can be used to plot error bars for RandomForest objects

    Parameters
    ----------
    forest : RandomForest
        Regressor or Classifier object.

    X : ndarray
        An array with shape (n_sample, n_features).

    inbag : ndarray (optional)
        The inbag matrix that fit the data. If set to `None` (default) it
        will be inferred from the forest. However, this only works for trees
        for which bootstrapping was set to `True`. That is, if sampling was
        done with replacement. Otherwise, users need to provide their own
        inbag matrix.

    calibrate: boolean (default True)
        Whether to apply calibration to mitigate Monte Carlo noise
        if calibrate = FALSE, some variance estimates may be negative
        due to Monte Carlo effects if the number of trees in rfrgr is too small
    
    speedup: boolean (default False)
        Whether to use interpolation to speedup calculation

    Returns
    -------
    An array with the unbiased sampling variance (V_IJ_unbiased)
    for a RandomForest object.

    See Also
    ----------
    :func:`calc_inbag`

    Notes
    -----
    The calculation of error is based on the infinitesimal jackknife variance,
    as described in [Wager2014]_ and is a Python implementation of the R code
    provided at: https://github.com/swager/randomForestCI

    .. [Wager2014] S. Wager, T. Hastie, B. Efron. "Confidence Intervals for
       Random Forests: The Jackknife and the Infinitesimal Jackknife", Journal
       of Machine Learning Research vol. 15, pp. 1625-1651, 2014.
    """
    
    ####### just to temporarily suppresssing warnings
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore")
    ####### end of suppresssing warnings
    
    
    if inbag is None:
        inbag = calc_inbag(X_train.shape[0], forest)

    pred = np.array([tree.predict(X_test) for tree in forest]).T
    pred_mean = np.mean(pred, 0)
    pred_centered = pred - pred_mean
    n_trees = forest.n_estimators
    V_IJ = _core_computation(X_train, X_test, inbag, pred_centered, n_trees)
    V_IJ_unbiased = _bias_correction(V_IJ, inbag, pred_centered, n_trees)

    # Correct for cases where resampling is done without replacement:
    if np.max(inbag) == 1:
        variance_inflation = 1 / (1 - np.mean(inbag)) ** 2
        V_IJ_unbiased *= variance_inflation

    if not calibrate:
        return V_IJ_unbiased

    if len(V_IJ_unbiased) <= 20:
        calibrate = False
        print("No calibration with n <= 20")
        return V_IJ_unbiased

    if calibrate:
        calibration_ratio = 2
        n_sample = np.ceil(n_trees / calibration_ratio)
        new_forest = copy.deepcopy(forest)
        new_forest.estimators_ = np.random.permutation(new_forest.estimators_)[:int(n_sample)]
        new_forest.n_estimators = int(n_sample)

        results_ss = random_forest_error(new_forest, X_train, X_test, calibrate=False)
        # Use this second set of variance estimates
        # to estimate scale of Monte Carlo noise
        sigma2_ss = np.mean((results_ss - V_IJ_unbiased)**2)
        delta = n_sample / n_trees
        sigma2 = (delta**2 + (1 - delta)**2) / (2 * (1 - delta)**2) * sigma2_ss

        # Use Monte Carlo noise scale estimate for empirical Bayes calibration
        if speedup:
            V_IJ_calibrated = calibrateEB(V_IJ_unbiased, sigma2, speedup=True)
        else:
            V_IJ_calibrated = calibrateEB(V_IJ_unbiased, sigma2)

        return V_IJ_calibrated


In [ ]:
# log usage

# LOG_PATH = r'P:\Proj\public\production_engineering\data_science\opalite_staging\log'
#if os.environ.get('MLTEMPLATE_LOG_PATH'):
#    LOG_PATH = os.environ.get('MLTEMPLATE_LOG_PATH')
LOG_PATH = os.environ.get('HOME')

logger = logging.getLogger('opalite')
logger.setLevel(logging.DEBUG)

logger.handlers = []
formatter = logging.Formatter('%(asctime)s, {}, %(message)s'.format(getpass.getuser()))
fh = logging.FileHandler(os.path.join(LOG_PATH, 'log.csv'), mode='a')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

logger.info('started')

## 0. Basic Settings

In [ ]:
# display settings

# widgets
row_layout = Layout(display='flex', flex_flow='row', align_items='center')
col_layout = dict(display='flex', flex_flow='column', justify_content='space-between', width='auto')

def leftm(m=10):
    '''Set left margin'''
    return '0px 0px 0px {}px'.format(m)

max_col = IntSlider(value=50, max=100, width='600px')
max_row = IntSlider(value=100, max=250, width='600px')

bs_row1 = Box([Label('Max Columns to Display:', layout=Layout(width='200px')),  max_col], layout=row_layout)
bs_row2 = Box([Label('Max Rows to Display:', layout=Layout(width='200px')),  max_row], layout=row_layout)
display(Box([bs_row1, bs_row2], layout=Layout(**col_layout)))

# function
pd.options.display.max_columns = max_col.value
pd.options.display.max_rows = max_row.value

def chg_setting(chg):
    '''Change basic settings.'''
    pd.options.display.max_rows = max_row.value
    pd.options.display.max_columns = max_col.value

max_col.observe(chg_setting, names='value')
max_row.observe(chg_setting, names='value')

## 1. Load Data
- accept .csv, .xls or .xlsx file (with single sheet) 
- data file path: it's using relative file path; by default it's the same directory as this notebook


In [ ]:
# load data

# widgets
# PATH = Text(description='Path:', width='800px', value='share')
filepath = Text(description='File Path:', value='')
loaddata_button = Button(description="Load Data", button_style='success', margin=leftm(650))
prog = FloatProgress(value=0, min=0, max=10, step=1, description='Progress:',
                     margin=leftm(550), width='250px')
display(filepath, loaddata_button, prog)

# functions
def loaddata(b):
    '''Load data.'''
    clear_output()
    prog.value = 0
    prog.description = 'Start...'
    filepath_ = filepath.value # os.path.join(PATH.value, filename.value)
    mld.read_data(filepath_)  # read data
    prog.value = 5
    prog.description = 'In Progress...'
    mld.save_current()  # save a copy of current snapshot
    prog.value = 10
    prog.description = 'Done!'
    mld.infer_categorical()
    clear_output()
    print('\n', 'Data Head')
    display(mld.head())  # display data head
    print('\n', 'Data Tail')
    display(mld.tail())  # display data tail
    print('\n', 'Summary Statistics')    
    display(mld.get_summary())  # display summary statistics
    logger.info('loaded data')
    
loaddata_button.on_click(loaddata)

## 2. Explore Data

- You can plot a subset of the data by giving filtering criterion like these:
    - currency == 'USD'
    - bondtype != 'CLO'
    - oas < 100 and oas > -50
    - sec_type == 'SUB' or sec_type == 'SENIOR'
    - sec_group not in ['CMO', 'CMBS']

In [ ]:
# Select Graph

# widgets
chart_select = ToggleButtons(
    options=['Pie Chart', 'Histogram', 'Box Plot', 'Cross Tab', 'Scatter Plot', 'Correlations'],
    tooltips=['Visualize single categorical variable', 'Visualize single numerical variable',
             'Visualize the relation between numerical and categorical variables',
             'Visualize the relation between two categorical variables',
             'Visualize pairwise distributions among a group of numerical variables',
             'Visualize pairwise correlations among a group of numerical variables'],
    value='Correlations', 
    margin=leftm()
)

display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px')))

# functions
grph = HTML('')

def switch_chart(chg):
    '''Switch between graph type.'''
    global grph
    clear_output()
    grph.close()
    grph = graph_mapping[chart_select.value]()
    display(grph)

chart_select.observe(switch_chart, names='value')

In [ ]:
# Pie chart

# widgets
def make_pie():
    '''create widgets for pie chart'''
    global pie_filter, pie_select, pie_limit, group_less_freq, pieb
    pie_filter = Text(value='', width='500px')
    pie_select = Dropdown(options=mld.get_header(sort=True), height='30px', width='200px',
                          margin=leftm())
    pie_limit = IntText(value=20, width='70px')
    group_less_freq = Checkbox(value=False)
    pieb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    
    row0 = Box([HTML('<h4>Pie Chart - Visualize Distributions of Categorical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), pie_filter, pieb], layout=row_layout)
    row2 = Box([Label('Select a Categorical Column: ', layout=Layout(width='200px')), pie_select,
                Label('Limit on # of Values: ', layout=Layout(width='200px'), margin=leftm(20)), pie_limit], layout=row_layout)
    row3 = Box([Label('Group Everything Below the Limit as "others": ', layout=Layout(width='400px')), group_less_freq],
                layout=row_layout)
    pie_chart = Box([row0, row1, row2, row3], layout=Layout(**col_layout),
                    height='160px')

    pie_select.observe(plot_pie, names='value')
    pieb.on_click(plot_pie)
    return pie_chart


# functions
def plot_pie(chg):
    '''plot the pie chart'''
    clear_output()
    if pie_filter.value.strip() != '':
        try:
            vc = mld.query_chain(pie_filter.value).ct_freq(pie_select.value, group_less_freq.value,
                                                           pie_limit.value)
            vc.astype('category').cat.remove_unused_categories(True)
        except Exception as e:
            print(e)
            vc = mld.ct_freq(pie_select.value, group_less_freq.value, pie_limit.value)
    else:
        vc = mld.ct_freq(pie_select.value, group_less_freq.value, pie_limit.value)
    trace = go.Pie(labels=list(vc.index), values=vc.values)
    figp = go.Figure(data=go.Data([trace]),
                     layout=dict(title='Break Down of {}'.format(pie_select.value.upper())))
    iplot(figp)
    logger.info('plotted pie')

In [ ]:
# Histogram

# widgets
def make_hist():
    '''make widgets for histogram'''
    global hist_filter, histb, hist_select, hist_min, hist_max, hist_bins
    hist_filter = Text(value='', width='500px')
    histb = Button(description="Plot", button_style='success', width='70px', height='30px',
                   margin=leftm(20))
    hist_select = Dropdown(options=mld.get_header(['int', 'float'], sort=True), height='30px',
                           width='200px', margin=leftm())
    hist_min = FloatText(value=float('-Inf'), width='70px')
    hist_max = FloatText(value=float('Inf'), width='70px')
    hist_bins = IntText(value=10, width='70px')
    
    row0 = Box([HTML('<h4>Histogram - Visualize Distributions of Numerical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), hist_filter, histb], layout=row_layout)
    
    row2 = Box([Label('Select a Numerical Column: ', layout=Layout(width='200px')), hist_select], layout=row_layout)
    row3 = Box([Label('Min: ', layout=Layout(width='200px')), hist_min, Label('Max: ', margin=leftm(20)), hist_max,
                Label('# of Bins: ', margin=leftm(20)), hist_bins], layout=row_layout)

    hist_chart = Box([row0, row1, row2, row3], layout=Layout(**col_layout), height='160px')
    
    hist_select.observe(plot_hist_new_col)
    histb.on_click(plot_hist)
    return hist_chart


# functions
def plot_hist_new_col(chg):
    '''plot histogram when switching columns'''
    hist_min.value = float('-Inf')
    hist_max.value = float('Inf')
    plot_hist(chg)
    
def plot_hist(chg):
    '''plot histogram'''
    clear_output()
    if hist_filter.value.strip() != '':
        x = mld.query(hist_filter.value)[hist_select.value]
    else:
        x = mld.data[hist_select.value]
    n = len(x)
    x = x[x > hist_min.value]
    x = x[x < hist_max.value]
    print('Percentage covered (after filtering): {:.2f}%'.format(len(x) / n * 100))
    if hist_min.value == float('-Inf'):
        hist_min.value = np.round(np.min(x), 2) - 1
    if hist_max.value == float('Inf'):
        hist_max.value = np.round(np.max(x), 2) + 1

    size = (hist_max.value - hist_min.value) / hist_bins.value
    trh = go.Histogram(x=x, histnorm='percent', marker=dict(color='rgb(0,0,100)'),
                       xbins=dict(start=hist_min.value - 0.5, size=size, end=hist_max.value + 0.5))

    layout = dict(bargap= 0.015, hovermode= 'x',
                  title='Histogram for {}'.format(hist_select.value.upper()),
                  yaxis= dict(title='Percentage (%)', autorange= True, showticklabels= True))
    figh = go.Figure(data=go.Data([trh]), layout=layout)
    iplot(figh)
    logger.info('plotted histogram')

In [ ]:
# Box plot

# widgets
def make_boxp():
    '''Make widgets for box plot.'''
    global box_filter, boxb, box_select_cat, box_select_num
    box_filter = Text(value='', width='500px')
    boxb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    box_select_cat = Dropdown(options=mld.get_header('category', sort=True), height='30px',
                              width='200px', margin=leftm())
    box_select_num = Dropdown(options=mld.get_header(['int', 'float'], sort=True), height='30px',
                              width='200px', margin=leftm())

    row0 = Box([HTML('<h4>Box Plot - Visualize Relations between Numerical and Categorical '
                     'Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), box_filter, boxb], layout=row_layout)
    row2 = Box([Label('Select y Axis: ', layout=Layout(width='200px')), box_select_num, 
                Label('Select x Axis: ', margin=leftm(20), layout=Layout(width='200px')),
                box_select_cat], layout=row_layout)
    box_chart = Box([row0, row1, row2], layout=Layout(**col_layout), height='120px')
    box_select_cat.observe(plot_box, names='value')
    box_select_num.observe(plot_box, names='value')
    boxb.on_click(plot_box)
    return box_chart


# functions
def plot_box(chg):
    '''Generate box plot.'''
    clear_output()
    if box_select_cat.value is not None and box_select_num.value is not None:
        temp_data = mld.data
        if box_filter.value.strip() != '':
            temp_data = mld.query(box_filter.value)
        names = list(temp_data[box_select_cat.value].unique())
        temp = []
        for name in names:
            mask = temp_data[box_select_cat.value] == name
            temp.append(go.Box(y=list(temp_data.ix[mask, box_select_num.value]), name=name, showlegend=False))
        figb = go.Figure(data=go.Data(temp),
                         layout=dict(title='Box Plot: {} by {}'.\
                                     format(box_select_num.value.upper(),
                                            box_select_cat.value.upper())))
        iplot(figb)
        logger.info('plotted box')

In [ ]:
# Cross Tab

# widgets
def make_xtab():
    '''Make widgets for cross tab plot.'''
    global xtab_filter, xtabb, xtab_select_x, xtab_select_y, xtab_pct
    xtab_filter = Text(value='', width='500px')
    xtabb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    xtab_select_x = Dropdown(options=mld.get_header('category', sort=True), height='30px',
                             width='200px', margin=leftm())
    xtab_select_y = Dropdown(options=mld.get_header('category', sort=True), height='30px',
                             width='200px', margin=leftm())
    xtab_pct = Checkbox(value=True)

    row0 = Box([HTML('<h4>Cross Tab - Visualize Relations between Two Categorical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), xtab_filter, xtabb], layout=row_layout)
    row2 = Box([Label('Select y Axis: ', layout=Layout(width='200px')), xtab_select_y, 
                Label('Select x Axis: ', margin=leftm(20), layout=Layout(width='200px')),
                xtab_select_x,], layout=row_layout)
    row3 = Box([Label('Show as percentage: ', layout=Layout(width='200px')), xtab_pct], layout=row_layout)
    xtab_chart = Box([row0, row1, row2, row3], layout=Layout(**col_layout), height='160px')
    
    xtab_select_x.observe(plot_xtab, names='value')
    xtab_select_y.observe(plot_xtab, names='value')
    xtabb.on_click(plot_xtab)
    return xtab_chart


# functions
def plot_xtab(chg):
    '''Generate cross tab.'''
    clear_output()
    x, y, normalize = xtab_select_x.value, xtab_select_y.value, xtab_pct.value

    if xtab_filter.value.strip() != '':
        temp_data = mld.query(xtab_filter.value)
    else:
        temp_data = mld.data
        
    temp_data[x].cat.remove_unused_categories(True)    
    temp_data[y].cat.remove_unused_categories(True)

    if normalize:
        temp = temp_data.groupby(x)[y].value_counts(normalize=True)
        vals = list(temp.index.levels[1])
        temp = (temp.unstack(1, fill_value=0) * 100).round(2).reset_index()
    else:
        temp = temp_data.groupby(x)[y].value_counts()
        vals = list(temp.index.levels[1])
        temp = temp.unstack(1, fill_value=0).round(2).reset_index()
        
    #vals = pd.Series(temp_data[y].unique()).dropna()
    data = []
    for val in vals:
        data.append(go.Bar(x=temp[x], y=temp[val], name=val))

    layout = go.Layout(barmode='stack')
    if normalize:
        layout.yaxis.title='Percentage (%)'
    else:
        layout.yaxis.title='Frequency'

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    logger.info('plotted cross-tab')

In [ ]:
# Scatter Plot

# widgets
def make_scatter():
    '''Make widgets for scatter plot.'''
    global sp_filter, spb, sp_height, sp_select, sp_ccol
    num_cols = mld.get_header(['int', 'float'], sort=True)
    sp_filter = Text(value='', width='500px')
    spb = Button(description="Plot", button_style='success', width='70px', height='30px',
                 margin=leftm(20))
    sp_height = np.min([300, np.max([len(num_cols) * 15, 100])])
    sp_select = SelectMultiple(options=num_cols, margin=leftm(),
                               height='{}px'.format(sp_height))
    sp_ccol = Dropdown(options=[None] + mld.get_header('category', sort=True), height='30px', width='200px', margin=leftm())

    row0 = Box([HTML('<h4>Scatter Plot - Visualize Pairwise Distributions among a Group of Numerical '
                     'Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), sp_filter, spb], layout=row_layout)
    row2 = Box([Label("Select Columns: ", layout=Layout(width='200px')), sp_select, Label("Group by: ", margin=leftm(30)), sp_ccol],
                layout=row_layout)
    scatter_chart = Box([row0, row1, row2], layout=Layout(**col_layout),
                        height='{}px'.format(sp_height + 110))
    spb.on_click(plot_scatter)
    return scatter_chart


# functions
def plot_scatter(chg):
    '''Generate scatter plot.'''
    clear_output()

    if sp_filter.value.strip() != '':
        temp_data = mld.query(sp_filter.value).copy(True)
    else:
        temp_data = mld.data.copy(True)
    
    sel_cols = list(sp_select.value)
    if sp_ccol.value is not None:
        temp_data[sp_ccol.value].cat.remove_unused_categories(True)
        temp_data[sp_ccol.value] = temp_data[sp_ccol.value].astype('object')
        sel_cols.append(sp_ccol.value)

    temp = temp_data[sel_cols]
    
    fig = FF.create_scatterplotmatrix(temp, index=sp_ccol.value, diag='histogram', height=800,
                                      width=800)
    iplot(fig)
    logger.info('plotted scatter')

In [ ]:
# Correlations

# widgets
def make_cor():
    '''Make widgets for correlations plot.'''
    global cor_filter, corb, cor_height, cor_select
    num_cols = mld.get_header(['int', 'float'], sort=True)
    cor_filter = Text(value='', width='500px')
    corb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    cor_height = np.min([300, np.max([len(num_cols) * 15, 100])])
    cor_select = SelectMultiple(options=num_cols, margin=leftm(),
                                height='{}px'.format(cor_height))
    row0 = Box([HTML('<h4>Correlations - Visualize Correlations among a Group of Numerical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), cor_filter, corb], layout=row_layout)
    row2 = Box([Label("Select Columns: ", layout=Layout(width='200px')), cor_select], layout=row_layout)
    cor_chart = Box([row0, row1, row2], layout=Layout(**col_layout),
                    height='{}px'.format(cor_height + 100))
    corb.on_click(plot_cor)
    return cor_chart


# functions
def plot_cor(chg):
    '''Generate correlations plot.'''
    clear_output()
    
    if cor_filter.value.strip() != '':
        temp_data = mld.query(cor_filter.value).copy(True)
    else:
        temp_data = mld.data.copy(True)
    
    sel_cols = list(cor_select.value)
    x, y = sel_cols, list(reversed(sel_cols))

    corr = temp_data[sel_cols].corr().round(2)
    z = np.flipud(corr.values)
    annotations = []
    for n, row in enumerate(z):
        for m, val in enumerate(row):
            var = z[n][m]
            annotations.append(
                dict(
                    text=str(val),
                    x=x[m], y=y[n],
                    xref='x1', yref='y1',
                    font=dict(color='black'),
                    showarrow=False)
                )

    colorscale = [[0, 'rgba(255,0,0,0.3)'], [0.5, 'rgba(255,255,0,0.3)'], [1, 'rgba(0,255,0,0.3)']]
    trace = go.Heatmap(x=x, y=y, z=z, zmin=-1, zmax=1, colorscale=colorscale, showscale=True)

    fig = go.Figure(data=[trace])
    fig['layout'].update(
        title="Correlations",
        annotations=annotations,
        xaxis=dict(ticks=''),
        yaxis=dict(ticks='', ticksuffix='  '),
        width=700,
        height=700,
        autosize=False
    )
    
    iplot(fig)
    logger.info('plotted correlation')

In [ ]:
graph_mapping = {'Pie Chart': make_pie, 'Histogram': make_hist, 'Box Plot': make_boxp,
                 'Cross Tab': make_xtab, 'Scatter Plot': make_scatter, 'Correlations': make_cor}

## 3. Clean Data 

In [ ]:
# Select Processing Function

# widgets
pros_select = ToggleButtons(
    options=['Drop Columns', 'Filter Rows', 'Handle Outlier', 'Convert Dates', 'Handle Missing Value',
             'Parse String'], value='Parse String', margin=leftm())

undob = Button(description='Undo Previous Action', button_style='info', width='160px')
torawb = Button(description='Restore to Original', button_style='warning', width='150px',
                margin=leftm())
savedatab = Button(description='Current Data -> CSV', button_style='success', width='150px',
                   margin=leftm(70))
savesumb = Button(description='Summary Stat -> CSV', button_style='success', width='160px',
                  margin=leftm())

pros = HTML('')

display(Box([undob, torawb, savedatab, savesumb],
            layout=Layout(display='flex', flex_flow='row', align_items='center',
                          margin='0px 0px 20px 0px')))
display(HBox([Label('Select: '), pros_select], layout=Layout(margin='0px 0px 30px 0px')))


# functions
def switch_pros(chg):
    '''Switch between processing functions.'''
    global pros
    clear_output()
    pros.close()
    pros = pros_mapping[pros_select.value]()
    display(pros)

def undo(b):
    '''Undo previous step.'''
    clear_output()
    mld.go_back()
    print('\nRestored data back to previous step.')
    print('\nSummary Statistics')
    display(mld.get_summary(['dtype', 'column name']))

def to_raw(b):
    '''Restore to original data.'''
    clear_output()
    mld.data = mld.data_snapshot.copy(deep=True)
    print('\nRestored data back to original.')
    print('\nSummary Statistics')
    display(mld.get_summary(['dtype', 'column name']))

def save_data(b):
    '''Save data to csv.'''
    clear_output()
    mld.data.to_csv('data.csv', index=False)
    print('Data are saved to {}'.format(os.path.join(os.getcwd(), 'data.csv')))

def save_summary(b):
    '''Save summary statistics to csv.'''
    clear_output()
    mld.get_summary().to_csv('data_summary.csv', index=False)
    print('Summary statistics are saved to {}'.format(os.path.join(os.getcwd(), 'data_summary.csv')))

pros_select.observe(switch_pros, names='value')
undob.on_click(undo)
torawb.on_click(to_raw)
savedatab.on_click(save_data)
savesumb.on_click(save_summary)

In [ ]:
# Drop columns

# widgets
def make_drop():
    '''make widgets for drop_columns'''
    global select_cols, drop_button
    select_cols = SelectMultiple(options=mld.get_header(sort=True), 
                                 height= '{}px'.\
                                 format(np.min([300, np.max([len(mld.data.columns) * 15, 100])])), 
                                 margin=leftm(20))
    drop_button = Button(description="Drop Columns", button_style='success',
                         margin="20px 0px 0px 50px")
    drop_button.on_click(drop_cols)
    select_cols.observe(display_selected_col, names='value')
    return HBox([Label("Select Columns", layout=Layout(width='200px')), select_cols, drop_button])


# functions
def display_selected_col(chg):
    clear_output()
    print('\nColumns to be dropped: ')
    print(select_cols.value, '\n')

def drop_cols(b):
    mld.drop_cols(select_cols.value)
    select_cols.options = mld.get_header(sort=True)
    clear_output()
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('drop columns')

In [ ]:
# Filter rows

# widgets
def make_filter():
    '''make widgets for filter_rows'''
    global filter_expr
    filter_expr = Text(width='500px')
    filterb = Button(description="Filter Rows", button_style='success', margin=leftm(650),
                     width='100px')
    row1 = Box([Label("Filter Rows by Criterion (result will persist): ", layout=Layout(width='400px')), filter_expr],
               layout=row_layout)
    row2 = Box([HTML(''' Examples:
                    <ul>
                        <li>currency == 'USD'</li>
                        <li>bondtype != 'CLO'</li>
                        <li>oas < 100 and oas > -50</li>
                        <li>sec_type == 'SUB' or sec_type == 'SENIOR'
                        <li>sec_group not in ['CMO', 'CMBS']
                    </ul>
                ''')], layout=row_layout)
    row3 = Box([filterb], layout=row_layout)
    filterb.on_click(filter_rows)
    return Box([row1, row2, row3], layout=Layout(**col_layout), height='190px')

# functions
def filter_rows(b):
    '''filter rows'''
    clear_output()
    print('\nBefore Transformation')
    print('Data Shape: {}'.format(mld.shape()))
    mld.query(filter_expr.value, inplace=True)
    print('\nAfter Transformation')
    print('Data Shape: {}'.format(mld.shape()))
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('filter rows')

In [ ]:
# Handle outliers

# widgets
def make_outlier():
    '''Make widgets for handle_outlier.'''
    global outlier_select, outlier_radio, outlier_lp, outlier_up
    outlier_select = Dropdown(options=mld.get_header(['float', 'int'], sort=True), height='30px',
                              width='200px', margin=leftm())
    outlier_radio = RadioButtons(value='Cap at Percentile',
                                 options=['Cap at Percentile', 'Remove Outliers'], margin=leftm())
    outlier_lp = FloatText(value=1, width='50px', margin=leftm())
    outlier_up = FloatText(value=99, width='50px', margin=leftm())
    outlierb = Button(description='Run', button_style='success', width='100px', margin=leftm(30))
    
    row1 = Box([Label('Select Column: ', layout=Layout(width='200px')), outlier_select, Label('Method: ', margin=leftm(30)),
                outlier_radio], layout=row_layout)
    row2 = Box([Label('Lower Percentile: ', layout=Layout(width='200px')), outlier_lp, 
                Label('Upper Percentile: ', margin=leftm(30), layout=Layout(width='200px')),
                outlier_up, outlierb], layout=row_layout)
    outlierb.on_click(handle_outlier)
    return Box([row1, row2], layout=Layout(**col_layout), height='100px')
    
# functions
def handle_outlier(b):
    '''Handle outliers.'''
    clear_output()
    method = 'capped' if outlier_radio.value == 'Cap at Percentile' else 'remove'
    col = outlier_select.value
    
    print('\nBefore Transformation')
    print('Data Shape: {}'.format(mld.shape()))
    print('Value Range of {}: ({}, {})'.format(col, mld.data[col].min(), mld.data[col].max()))
    mld.handle_outlier(col, method, outlier_lp.value, outlier_up.value, True)
    print('\nAfter Transformation')
    print('Data Shape: {}'.format(mld.shape()))
    print('Value Range of {}: ({}, {})'.format(col, mld.data[col].min(), mld.data[col].max()))
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('handle outlier')

In [ ]:
# Convert date columns

# widgets
def make_pdate():
    '''Make widgets for converting dates.'''
    global pdate_select, pdateb, pdate_to, pdate_pattern, pdate_toggle, pdateb, pdateb2, pdate_ref, \
        col1, col21, col22, pdate_unit
    date_cols = mld.get_header('datetime', sort=True)
    pdate_height = np.min([300, np.max([len(date_cols) * 15, 200])])
    pdate_select = SelectMultiple(options=list(sorted(date_cols)), height='{}px'.format(pdate_height))
    pdate_to = Dropdown(height='30px', width='200px', margin=leftm(),
                        options=['year', 'month', 'day', 'hour', 'minute', 'second', 'dayofyear',
                                 'weekofyear', 'days_in_month', 'date', 'time', 'weekday',
                                 'weekday_name', 'quarter', 'is_month_start', 'is_month_end',
                                 'is_quarter_start', 'is_quarter_end', 'str'])
    pdate_pattern = Text(value='%Y-%m-%d', width='150px', margin=leftm())
    pdateb = Button(description='Convert Dates', button_style='success')
    pdateb2 = Button(description='Compute Difference', button_style='success')
    
    pdate_ref = Dropdown(options=['today'] + date_cols, value='today', height='30px', width='200px',
                         margin=leftm())
    pdate_toggle = ToggleButtons(value='Convert', options=['Convert', 'Date Diff'])
    pdate_unit = Dropdown(options=['D', 'h', 'm', 's', 'ms'], value='D', height='30px',
                          width='100px', margin=leftm())
    
    col1 = VBox([Label("Select Date Columns: "), pdate_select])
    
    col21 = VBox([HBox([pdate_toggle], margin='0px 0px 20px 0px'),
                 HBox([Label("Convert to: ", layout=Layout(width='200px')), pdate_to], margin='0px 0px 20px 0px'),
                 HBox([Label("Regex Pattern (only for 'str'): ", layout=Layout(width='200px')), pdate_pattern],
                 margin='0px 0px 20px 0px'),
                 pdateb], layout=Layout(margin=leftm(30), justify_content='flex-end',
                                    height=pdate_select.height, display='flex'))
    
    col22 = VBox([HBox([pdate_toggle], margin='0px 0px 20px 0px'),
                  HBox([Label("Reference Date: ", layout=Layout(width='200px')), pdate_ref], margin='0px 0px 20px 0px'),
                  HBox([Label("Units: "), pdate_unit], margin='0px 0px 20px 0px'),
                  pdateb2], layout=Layout(margin=leftm(30), justify_content='flex-end',
                                    height=pdate_select.height, display='flex'))
    
    pdateb.on_click(convert_date)
    pdateb2.on_click(date_diff)
    pdate_toggle.observe(redisplay_pdate, names='value')
    return HBox([col1, col21])
    
def redisplay_pdate(chg):
    '''Switch between date_convert and date_diff'''
    global pros
    pros.close()
    if pdate_toggle.value == 'Convert':
        pros = HBox([col1, col21])
    else:
        pros = HBox([col1, col22])        
    display(pros)


# function
def convert_date(b):
    '''Convert dates.'''
    clear_output()
    new_names = ['{}.{}'.format(name, pdate_to.value) for name in list(pdate_select.value)]
    mld.data[new_names] = mld.convert_dates(list(pdate_select.value), to=pdate_to.value,
                                            pattern=pdate_pattern.value, inplace=False)
    if pdate_to.value == 'str':
        mld.infer_categorical(new_names)
    print('\nBefore Transformation')
    print('Data Types:\n{}'.format(mld.data[list(pdate_select.value)].dtypes.reset_index().values))
    display(mld.data[list(pdate_select.value)].head())
    print('After Transformation - new columns are created: ')
    print('Data Types:\n{}'.format(mld.data[new_names].dtypes.reset_index().values))
    display(mld.data[new_names].head())
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))

def date_diff(b):
    '''Take date diff'''
    clear_output()
    ref_date = pdate_ref.value
    new_names = ['{} - {}'.format(name, ref_date) for name in list(pdate_select.value)]
    if ref_date == 'today':
        ref_date = datetime.datetime.today()
    mld.data[new_names] = mld.date_diff(pdate_select.value, ref_date, pdate_unit.value, inplace=False)
    print('\nBefore Transformation')
    display(mld.data[list(pdate_select.value)].head())
    print('After Transformation - new columns are created: ')
    display(mld.data[new_names].head())
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('convert date')

In [ ]:
# Handle missing value

# widgets
def make_miss():
    global fillna_num, fillna_cat, fillna_group, fillnab, dropna_axis, dropna_thrh, dropnab
    fillna_num = Dropdown(options=['median', 'mean', 'zero'], value='median', height='30px',
                          width='150px', margin=leftm())
    fillna_cat = Text(value='unknown', width='150px', margin=leftm(15))
    fillna_group = Dropdown(height='30px', width='200px', margin=leftm(),
                            options=[None] + mld.get_header(['category', 'object']))
    fillnab = Button(description="Fill Nulls", button_style='success', margin=leftm(280), width='100px')
    fillnab.on_click(fillna)

    dropna_axis = RadioButtons(value='columns', options=['rows', 'columns'], margin=leftm())
    dropna_thrh = FloatText(value=90, width='100px', margin=leftm())
    dropnab = Button(description="Drop Nulls", button_style='success', margin=leftm(130), width='100px')
    dropnab.on_click(dropna)
    
    row1 = Box([Label('Drop: ', layout=Layout(width='200px')), dropna_axis, 
                Label('when percent of nulls > ', layout=Layout(width='200px'), margin=leftm(20)),
                dropna_thrh, dropnab], layout=row_layout)    
    row2 = Box([Label('Otherwise fill with:', layout=Layout(width='200px'))], layout=row_layout)
    row3 = Box([Label('Numerical Fill: ', layout=Layout(width='200px')), fillna_num,
                Label('Numerical Fill Group by: ', margin=leftm(20), layout=Layout(width='200px')), fillna_group], layout=row_layout)
    row4 = Box([Label('Category Fill: ', layout=Layout(width='200px')), fillna_cat, fillnab], layout=row_layout)

    return Box([row1, row2, row3, row4], layout=Layout(**col_layout), height='180px')
    

# function
def fillna(b):
    clear_output()
    mld.fill_na(cat_fill=fillna_cat.value, num_fill=fillna_num.value, groupby=fillna_group.value)
    display(mld.get_summary('dtype'))
    logger.info('fill null')
    
def dropna(b):
    clear_output()
    print('\nData Shape Before: {}'.format(mld.shape()))
    mld.drop_na(axis=int(dropna_axis.value == 'columns'), threshold=1 - dropna_thrh.value / 100)
    print('Data Shape After: {}'.format(mld.shape()))
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('drop null')

In [ ]:
# Parse string value

def make_pstr():
    global pstr_select, pstr_method, pstr_mtdp, pstrb
    cols = mld.get_header(['category', 'object'])
    pstr_height = str(np.min([300, np.max([len(cols) * 15, 200])])) + "px"
    pstr_select = SelectMultiple(options=cols, height='{}px'.format(pstr_height))
    pstr_method = Dropdown(height='30px', width='150px', margin=leftm())
    pstr_method.options=['capitalize', 'contains', 'count', 'endswith', 'extract',
                        'find', 'findall', 'join', 'len', 'lower', 'repeat',
                         'slice', 'split', 'startswith', 'strip', 'upper',
                        'isalnum', 'isalpha', 'isdigit', 'isspace', 'islower', 'isupper', 'isnumeric',
                        'isdecimal']
    pstr_mtdp = Text(width='150px', margin=leftm())
    pstrb = Button(description="Parse String", button_style='success', width='100px')
    pstrb.on_click(pstr)
    
    col1 = VBox([Label("Select Columns: ", layout=Layout(width='200px')), pstr_select])
    col2 = VBox([HBox([Label('Select String Method: ',layout=Layout(width='200px')), pstr_method], margin='0px 0px 40px 0px'),
                 HBox([Label('Additional Input: ', layout=Layout(width='200px')), pstr_mtdp],
                      margin='0px 0px 50px 0px'), pstrb],
                layout=Layout(margin=leftm(30), justify_content='flex-end', height=pstr_height,
                              display='flex'))
    return HBox([col1, col2])


def pstr(b):
    clear_output()
    
    if pstr_method.value in ['split', 'findall']:
        new_names = []
        for col in pstr_select.value:
            res = mld.data[col].astype(str).str.split(pstr_mtdp.value, expand=True)
            names = ['{}.{}'.format(col, i) for i in res.columns]
            mld.data[names] = res
            new_names += names
    else:
        new_names = ['{}.{}'.format(name, pstr_method.value) for name in pstr_select.value]
        if len(pstr_mtdp.value.strip()) > 0:
            mtdp = pstr_mtdp.value
            try:
                mtdp = int(mtdp)
            except:
                pass
            res = mld.parse_str(list(pstr_select.value), pstr_method.value, True, False, mtdp)
            
        else:
            res = mld.parse_str(list(pstr_select.value), pstr_method.value)
        
        if len(res.shape) == 1:
            mld.data[new_names[0]] = res
        else:
            mld.data[new_names] = res
        
    print('\nBefore Transformation')
    display(mld.data[list(pstr_select.value)].head())
    print('After Transformation - new columns are created: ')
    display(mld.data[new_names].head())
    print('\nSummary Statistics')
    display(mld.get_summary('dtype'))
    logger.info('parse string')

In [ ]:
pros_mapping = {'Drop Columns': make_drop, 'Filter Rows': make_filter, 'Convert Dates': make_pdate,
                'Handle Missing Value': make_miss, 'Parse String': make_pstr,
                'Handle Outlier': make_outlier}

## 4. Build Model

In [ ]:
# override ipythonwidgets build in function 

def _get_min_max_value(min, max, value=None, step=None):
    """Return min, max, value given input values with possible None."""
    if value is None:
        if not max > min:
            raise ValueError('max must be greater than min: (min={0}, max={1})'.format(min, max))
        diff = max - min
        value = min + (diff / 2)
        # Ensure that value has the same type as diff
        if not isinstance(value, type(diff)):
            value = min + (diff // 2)
    elif min is None and max is None:
        # if not isinstance(value, Real):
        #     raise TypeError('expected a real number, got: %r' % value)
        if not value:
            t = type(value)
            min, max = (t(0), t(1))
        elif value > 0:
            min, max = (0, np.max([10*value, 10]))
        else:
            min, max = (np.min([10*value, -10]), 0)
    else:
        raise ValueError('unable to infer range, value from: ({0}, {1}, {2})'.format(min, max, value))
    if step is not None:
        # ensure value is on a step
        r = (value - min) % step
        value = value - r
    return min, max, value

interaction._get_min_max_value = _get_min_max_value

In [ ]:
# Run single model

model_options = {'LinearRegression': 'ols',
            'Ridge': 'ridge',
            'Lasso': 'lasso',
            'ElasticNet': 'enet',
            'BayesianRidge': 'bayes',
#             'RANSACRegressor': 'ransac',
            'SVR': 'svr',
            'KNeighborsRegressor': 'knnrgr',
            'DecisionTreeRegressor': 'dtrgr',
            'AdaBoostRegressor': 'ab',
            'GradientBoostingRegressor': 'gbrgr',
            'RandomForestRegressor': 'rfrgr',
            'ExtraTreesRegressor': 'ert',
            'BaggingRegressor': 'bag'}

model_args = {
    'ols': {'fit_intercept': True, 'normalize': False, 'n_jobs': 1},
    'ridge': {'alpha': 1.0, 'fit_intercept': True, 'normalize': False},
    'lasso': {'alpha': 1.0, 'fit_intercept': True, 'normalize': False},
    'enet': {'alpha':1.0, 'l1_ratio': 0.5, 'fit_intercept': True, 'normalize': False},
    'bayes': {'alpha_1': 1e-06, 'alpha_2': 1e-06, 'lambda_1': 1e-06, 'lambda_2': 1e-06, 
              'compute_score': False, 'fit_intercept': True, 'normalize': False},
#     'ransac': {'max_trials': 100, 'stop_probability': 0.99},
    'svr': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'], 'degree': 3,
            'gamma': 'auto', 'coef0': 0.0, 'C': 1.0, 'epsilon': 0.1,'shrinking': True},
    'knnrgr': {'n_neighbors': 5, 'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size': 30, 'p': 2, 'metric': 'minkowski', 'n_jobs': 1},
    'dtrgr': {'criterion': 'mse', 'splitter': 'best', 'max_depth': 5, 'min_samples_split': 10,
           'min_samples_leaf': 15, 'min_weight_fraction_leaf': 0.0},
    'ab': {'n_estimators': 50, 'learning_rate': 1.0, 'loss': 'linear'},
    'gbrgr': {'loss': 'ls', 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0,
           'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0,
           'max_depth': 3, 'alpha': 0.9},
    'rfrgr': {'n_estimators': 10, 'criterion': 'mse', 'max_depth': 20, 'min_samples_split': 2,
           'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto',
           'bootstrap': True, 'oob_score': False, 'n_jobs': 1},
    'ert': {'n_estimators': 10, 'criterion': 'mse', 'max_depth': 20, 'min_samples_split': 2,
            'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto',
            'bootstrap': False, 'oob_score': False, 'n_jobs': 1},
    'bag': {'n_estimators': 10, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap': True,
            'bootstrap_features': False, 'oob_score': False, 'n_jobs': 1}}

metric_options = {'R2 Score': 'r2', 'Mean Squared Error': 'mse', 'Mean Absolute Error': 'mae', 
                  'Median Absolute Error': 'meae', 'Explained Variance Score': 'evs'}

# widgets
model_dropcat = Checkbox(value=True, margin=leftm(), width='50px')
model_dropthrh = IntSlider(value=500, max=500, margin=leftm())

model_seed = IntText(value=0, width='150px', margin=leftm())
model_split = FloatSlider(value=0.7, min=0, max=1, step=0.01, width='600px', margin=leftm())
model_select = Dropdown(options=model_options, value='dtrgr', height='30px', margin=leftm(),
                        width='200px')

model_metric = SelectMultiple(options=metric_options, value=('r2',), height='90px', margin=leftm(),
                              width='200px')

model_prog = FloatProgress(value=0, min=0, max=10, step=1, description='Progress:',
                           margin=leftm(600), width='200px')

modelb = Button(description="Run Model", button_style='success', width='100px', margin=leftm(150))
model_save = Button(description="Save Results", button_style='warning', width='100px', margin=leftm(150))

def make_single():
    '''Make widgets for single model.'''
    global model_target, model_features
    
    mlm.define_regressor(model_name=model_select.value, **model_args[model_select.value])
    print(mlm.model)
    
    model_target = Dropdown(options=mld.get_header(['float', 'int'], sort=True), height='30px',
                            margin=leftm(), width='200px')

    columns = mld.get_header(['float', 'int', 'category', 'object'], sort=True)
    features_height = np.min([300, np.max([len(columns) * 12, 100])])
    model_features = SelectMultiple(options=columns, margin=leftm(),
                                    height='{}px'.format(features_height))

    rows = []
    rows.append(Box([Label('Target Variable: ', layout=Layout(width='200px')), model_target], layout=row_layout))
    rows.append(Box([Label('Independent Variables: ', layout=Layout(width='200px')), model_features], layout=row_layout))
    rows.append(Box([Label('Evaluation Metric: ', layout=Layout(width='200px')), model_metric], layout=row_layout))
    rows.append(Box([Label('Select Model: ', layout=Layout(width='200px')), model_select, model_save, modelb], layout=row_layout))
    rows.append(Box([model_prog], layout=row_layout))
    rows.append(Box([model_toggle_param], margin='0px 0px 20px 0px'))

    return Box(rows, layout=Layout(**col_layout), height='{}px'.format(300 + features_height))

# functions
def model_chg(chg):
    '''Listen to model change event, and re-define regressor and create model parameter Box.'''
    global model_param_displayed, w
    clear_output()
    w.close()
    mlm.define_regressor(model_name=model_select.value)
    print(mlm.model)
    
    args = model_args[model_select.value]
    w = interactive(set_params, **args)
    w = modify_box_widget(w)
    if model_param_displayed:
        display(w)

def set_params(**args):
    '''Set model parameters and print.'''
    mlm.model.set_params(**args)
    print(mlm.model)

def modify_box_widget(w):
    '''Modify the model parameter box widget.'''
    children = []
    for child in w.children:
        child.margin = leftm()
        if hasattr(child, 'description'):
            desc = child.description
            child.description = ''
            children.append(Box([Label('{}: '.format(desc), layout=Layout(width='200px')), child], layout=row_layout))
        else:
            children.append(child)
    return Box(children, layout=Layout(**col_layout), height='{}px'.format(40 * len(w.children)))

model_select.observe(model_chg, names='value')

In [ ]:
# Run Multiple Model

# widgets
model_select2 = SelectMultiple(options=model_options, height='200px', margin=leftm(),
                               width='200px')

model_metric2 = Dropdown(options=metric_options, value='r2', height='30px', margin=leftm(),
                         width='200px')

model_toggle_param = Button(description="Toggle Model Settings", button_style='success',
                            width='170px')
    
def make_multiple():
    '''Make widgets for multiple models'''
    global model_target, model_features

    #mlm.define_regressor(model_name='all')
    mlm.define_regressor(model_name='run_all_regressors')
    model_target = Dropdown(options=mld.get_header(['float', 'int'], sort=True), height='30px',
                            margin=leftm(), width='200px')

    columns = mld.get_header(['float', 'int', 'category', 'object'], sort=True)
    features_height = np.min([300, np.max([len(columns) * 12, 100])])
    model_features = SelectMultiple(options=columns, margin=leftm(),
                                    height='{}px'.format(features_height))

    rows = []
    rows.append(Box([Label('Target Variable: ', layout=Layout(width='200px')), model_target], layout=row_layout))
    rows.append(Box([Label('Independent Variables: ', layout=Layout(width='200px')), model_features], layout=row_layout))
    rows.append(Box([Label('Evaluation Metric: ', layout=Layout(width='200px')), model_metric2], layout=row_layout))
    rows.append(Box([Label('Select Models: ', layout=Layout(width='200px')), model_select2], layout=row_layout))
    rows.append(Box([modelb], margin=leftm(500)))
    rows.append(Box([model_prog], layout=row_layout))
    rows.append(model_toggle_param)

    return Box(rows, layout=Layout(**col_layout), height='{}px'.format(450 + features_height))


# functions
model_param_displayed = False
def toggle_param(b):
    '''Toggle model parameters.'''
    global model_param_displayed, w

    if model_param_displayed:
        w.close()
    else:
        if mode_select.value == 'Run Single Model':
            args = model_args[model_select.value]
            w = interactive(set_params, **args)
            w = modify_box_widget(w)
        else:
            w = get_multiple_params()
        display(w)
    model_param_displayed = not model_param_displayed

def get_multiple_params():
    '''Create parameter Box for multiple models.'''
    rows = []
    for model in model_select2.value:
        args = model_args[model]
        w = interactive(set_params_model, model=fixed(model), **args)
        w = modify_box_widget(w)
        name = re.match('(\w+)\(' , str(mlm.models[model])).group(1)
        rows.append(HTML('<b>{}</b>'.format(name), margin='20px 0px 10px 0px'))
        rows.append(w)
    return VBox(rows)

def set_params_model(model, **args):
    '''Set parameters for a specific model and print.'''
    mlm.models[model].set_params(**args)
    print(mlm.models[model])

model_toggle_param.on_click(toggle_param)
  

def run_model(b):
    clear_output()
    
#     perf_log(client='ADS-OPAL', module='Opal-regression', submodule='build_model', tag_def='version-1.0', user=user)
    
    if len(model_features.value) == 0:
        raise ValueError("Please select independent variables.")
    
    model_prog.value = 0
    model_prog.description = 'Start...'
    features = list(np.setdiff1d(model_features.value, [model_target.value]))
    all_cols = features + [model_target.value]
    if model_dropcat.value:
        mld.drop_cat_cols(all_cols, threshold=model_dropthrh.value)
        all_cols = list(np.intersect1d(all_cols, mld.get_header()))

    if mld.data[all_cols].isnull().sum().sum() > 0:
        mld.fill_na(all_cols)

    mld.create_dummy_data(features)
    
    global XX_train, yy_train, XX_test, yy_test

    XX_train, yy_train, XX_test, yy_test = \
        mlm.split_random(mld.dummies, mld.data[model_target.value],
                         train_ratio=model_split.value)
    

    if mode_select.value == 'Run Single Model':
        mlm.fit_model(XX_train, yy_train, indep_cols=features) 
        
        pred_name = "predicted_" + model_target.value
        mld.data[pred_name] = mlm.predict(mld.dummies)
        pred_error_name = "error_predicted_" + model_target.value
        mld.data[pred_error_name] = mlm.predict(mld.dummies) - mld.data[model_target.value]
      
        model_prog.value = 5
        model_prog.description = 'Progress.'
        #show_results(X_train, y_train, X_test, y_test)
        model_prog.description = 'Done!'
        model_prog.value = 10
        logger.info('ran single model')
    else:
        model_prog.value = 5
        model_prog.description = 'Progress.'
        res = mlm.model_comparison(XX_train, yy_train, XX_test, yy_test, features,
                                   list(model_select2.value))
        res = pd.Series(res).reset_index().round(4)
        res.columns = ['Model', model_metric2.value]
        res.sort_values(model_metric2.value, ascending=False, inplace=True)
        print('\nModel Performance')
        display(res)
        model_prog.description = 'Done!'
        model_prog.value = 10
        logger.info('ran multiple models')

def show_results(X_train, y_train, X_test, y_test):
    '''Show prediction results and performance scores.'''
    global train_scores, test_scores, fi, coef 

    # performance score
    print("performance score")
    print("On Training Set")
    pred_train = mlm.predict(X_train)
    #train_scores = mlm.eval_model(y_train, pred_train, metric=model_metric.value)
    train_scores = mlm.eval_model(ytrue=y_train, ypred=pred_train, metric_list=model_metric.value)
    train_scores = pd.DataFrame({'metric': model_metric.value, 'score': train_scores}).round(4)
    display(train_scores)

    print("On Testing Set")
    pred_test = mlm.predict(X_test)
    #test_scores = mlm.eval_model(y_test, pred_test, metric=model_metric.value)
    test_scores = mlm.eval_model(ytrue=y_test, ypred=pred_test, metric_list=model_metric.value)
    test_scores = pd.DataFrame({'metric': model_metric.value, 'score': test_scores}).round(4)
    display(test_scores)

    if hasattr(mlm.model, 'feature_importances_'):
        fi = mlm.check_feature_importance(group=True).round(4)
        print('\nFeature Importance')
        display(fi)

    if model_select.value == 'dtrgr':
       img_bi = mlm.decision_tree_plot()
       display(SVG(img_bi))

    if model_select.value == 'ols':
        coef = pd.DataFrame({'Feature': mlm.dummy_indep_cols, 'Coef': mlm.model.coef_})
        pos_mask = coef['Coef'] > 0
        pos_coef = coef[pos_mask].sort_values('Coef', ascending=False)
        neg_coef = coef[~pos_mask].sort_values('Coef')
        display('Positive Coefficients', pos_coef)
        display('Negative Coefficients', neg_coef)

    # make prediction
    print("Predictions on Test Data")
    display(pd.concat([pred_test, y_test, mld.data.ix[y_test.index, mlm.indep_cols]], axis=1))

def save_results(b):
    try:
        if not os.path.exists('results'):
            os.makedirs('results')
        train_scores.to_csv('results/train_scores.csv', index=False)
        test_scores.to_csv('results/test_scores.csv', index=False)
        fi.to_csv('results/feature_importance,csv', index=False)
        if model_select.value == 'ols':
            coef.to_csv('results/coefficients.csv')
        elif model_select.value == 'dtrgr':
            png = mlm.decision_tree_plot('png')
            with open('results/decision_tree_plot.png', 'wb') as f:
                f.write(png)
        print('Done!')
    except Exception as e:
        print(str(e))

modelb.on_click(run_model)
model_save.on_click(save_results)

In [ ]:
# Select Mode

mode_select = ToggleButtons(options=['Run Single Model', 'Run Multiple Models'], value='Run Multiple Models', margin=leftm())

display(HBox([Label('Mode: '), mode_select], layout=Layout(margin='0px 0px 30px 0px')))


mode_mapping = {'Run Single Model': make_single, 'Run Multiple Models': make_multiple}
mode = HTML('')
w = HTML('')

rows = []
rows.append(Box([Label('Drop Categorical Column if Too Many Distinct Values: ', layout=Layout(width='600px')), model_dropcat,
                 Label('Threshold: '), model_dropthrh], layout=row_layout))
rows.append(Box([Label('Random Seed: ', layout=Layout(width='200px')), model_seed], layout=row_layout))
rows.append(Box([Label('Proportion for Training: ', layout=Layout(width='200px')), model_split], layout=row_layout))

display(Box(rows, layout=Layout(**col_layout), height='120px', margin='0px 0px 10px 0px'))

def switch_mode(chg):
    '''Switch between modeling mode.'''
    global mode
    clear_output()
    mode.close()
    mode = mode_mapping[mode_select.value]()
    display(mode)

mode_select.observe(switch_mode, names='value')

In [ ]:
#ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
#sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


In [ ]:
#sys.getsizeof(globals().get(MLData))
#dir()

## 5. Model Result (only for single model)
- model result in testing data set:

In [ ]:
# Select Graph

# widgets
Result_chart_select = ToggleButtons(
    options=['Visualize Prediction', 'Display Predictions'],
    tooltips=['Visualize Prediction outside training set, Confidence Interval and real values', 'Display Predictions'],
    value='Visualize Prediction', 
    margin=leftm()
)

display(HBox([Label('Model Result Type: ', layout=Layout(width='200px')), Result_chart_select], layout=Layout(margin='0px 0px 30px 0px')))

# functions
grph = HTML('')

def Result_switch_chart(chg):
    '''Switch between graph type.'''
    global grph
    clear_output()
    if (grph != None):
        grph.close()
    #grph.close()
    grph = Result_graph_mapping[Result_chart_select.value]()
    display(grph)

Result_chart_select.observe(Result_switch_chart, names='value')

In [ ]:
# Confidence Interval chart

# widgets
def make_Confidence_Interval():
    '''create widgets for Confidence_Interval'''
    #global pie_filter2, pie_select2, pie_limit2, group_less_freq2, pieb2
    
    global pie_select2, pieb2

    #pie_filter2 = Text(value='', width='500px')
    pie_select2 = Dropdown(options=mld.get_header(sort=True), height='30px', width='200px',
                          margin=leftm())
    #pie_limit2 = IntText(value=20, width='70px')
    #group_less_freq2 = Checkbox(value=False)
    pieb2 = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    
    row0 = Box([HTML('<h4> Visualize outsample prediction and confidence interval</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([pieb2], layout=row_layout)

    scatter_chart = Box([row0, row1], layout=Layout(**col_layout),
                    height='160px')

    pie_select2.observe(plot_Confidence_Interval, names='value')
    pieb2.on_click(plot_Confidence_Interval)
    return scatter_chart

# functions
def gradientBoostingRegr_band(clf_outside, X_train, y_train, X_test, y_test):
    '''clf must be gradient boosting regressor'''
    ## for upper bound
    clf = clone(clf_outside)
    alpha = 0.95
    clf.set_params(loss='quantile')
    clf.set_params(alpha=alpha)
    clf.fit(X_train, y_train)
    y_upper = clf.predict(X_test)
    
    ## for lower bound
    clf.set_params(alpha=1.0 - alpha)
    clf.fit(X_train, y_train)
    y_lower = clf.predict(X_test)
    
    ## for prediction
    clf.set_params(loss='ls')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    return y_pred, y_upper, y_lower
    
    
def plot_Confidence_Interval(chg):
    '''plot the Confidence Interval chart'''
    clear_output()
    if ((type(mlm.model) != sklearn.ensemble.forest.RandomForestRegressor) and (type(mlm.model) != sklearn.ensemble.gradient_boosting.GradientBoostingRegressor)):
        return None
    
    forest = mlm.model
    #X_train, y_train, X_test, y_test = \
    #    mlm.split_random(mld.dummies, mld.data[model_target.value],
    #                     train_ratio=model_split.value)
               
    
    if (type(forest) == sklearn.ensemble.forest.RandomForestRegressor):
        #forest.fit(X_train, y_train)
        y_hat = forest.predict(XX_test)
        #V_IJ_calibrated = fci.random_forest_error(forest,X_train, X_test)
        V_IJ_calibrated = random_forest_error(forest,XX_train, XX_test)

        yerr=np.sqrt(V_IJ_calibrated)
        y_mid = y_hat

    if (type(forest) == sklearn.ensemble.gradient_boosting.GradientBoostingRegressor):
        y_hat, y_upper, y_lower = gradientBoostingRegr_band(clf_outside=mlm.model, X_train=XX_train, 
                                                            y_train=yy_train, X_test=XX_test, y_test=yy_test)
        y_mid = 0.5 * (y_upper + y_lower)
        yerr = y_upper - y_mid
            
    confidence_band = go.Scatter(
                x=list(yy_test),
                y=list(y_mid),
                error_y=dict(
                    type='data',
                    array=list(yerr),
                    visible=True
                ),
                mode='markers',
                name='confidence_band'
            )

    real_value = go.Scatter(
                x=list(yy_test),
                y=list(yy_test),
                mode='markers',
                name='real_value'
            )
    
    prediction = go.Scatter(
                x=list(yy_test),
                y=list(y_hat),
                mode='markers',
                name='prediction'
            )
        
    data = [confidence_band, real_value, prediction]
    
    layout = go.Layout(xaxis=dict(title='ytest'),
                   yaxis=dict(title='ypred')
                   )

    figp = go.Figure(data=data, layout=layout)
    #figp = go.Figure(data=[trace0, trace1, fx, observations], layout=layout)
    
    iplot(figp)
    logger.info('plotted Confidence Interval')

In [ ]:
def display_predictions_on_test():    
    # make prediction
    #print("Predictions on Test Data")
    forest = mlm.model
    #X_train, y_train, X_test, y_test = \
    #    mlm.split_random(mld.dummies, mld.data[model_target.value],
    #                     train_ratio=model_split.value)
    
    #show_results(X_train=XX_train, y_train=yy_train, X_test=XX_test, y_test=yy_test)
    
    try:
        show_results(X_train=XX_train, y_train=yy_train, X_test=XX_test, y_test=yy_test)
    except:
        print("Capture error: need to run model first, and Graphviz executables are on your systems' path")
    #display(pd.concat([pred_test, y_test, mld.data.ix[y_test.index, mlm.indep_cols]], axis=1))


In [ ]:
Result_graph_mapping = {'Visualize Prediction': make_Confidence_Interval, 'Display Predictions': display_predictions_on_test}

## 6. What's Behind the Scene

Take a look at the Python script (click "show/hide code" and see the first code block). You will see 

```python
from mltemplate.mldata import MLData
from mltemplate.mlmodel import MLModel
mld = MLData()
mlm = MLModel()
```

Here we created 'mld' as an instance of the MLData class, which is responsible for data cleaning, parsing and transforming, etc, while 'mlm' is an instance of the MLModel class, which is responsible for modeling building, predicting, and evaluating model performance, etc.

For MLData you can try some basic usages like

```python
print(mld.head(20))  # check the top 20 rows
print(mld.get_header())  # check column headers
print(mld.ct_freq('col_A'))  # count frequencies for each unique value in col_A
print(mld.get_summary())  # get the summary statistics
print(mld.query('col_A < 5'))  # filter rows by sql-like criterion
```

For MLModel you can try things like

```python
mlm.define_regressor(model_name='dtrgr')  # set the regressor to be a decision tree model
mlm.fit_model(X, y, indep_cols=['colA', 'colB', 'colC'])  # fit a model using X as the input data, y as target variable, and 'colA', 'colB', 'colC' as the independent columns.
mlm.predict(X)  # make predictions using X as the input data
mlm.eval_model(ytrue, ypred, metric='r2')  # evaluate model performance by comparing the true target 'ytrue' with the prediction 'ypred', using R-squared as the metric.
```

## Appendix: Some Keyboard Shortcuts:

- **ctrl + shift + p**: bring up the entire keyboard shortcut list 

- In command mode (when selected cell border is in <span style='color:blue'>**blue**</span>)
    - **enter**: enter edit mode
    - **esc**: exit edit mode
    - **ctrl-enter**: execute the current cell
    - **b**: insert a cell below
    - **a**: insert a cell above
    - **dd**: delete current cell


- In edit model (when selected cell border is in <span style='color:green'>**green**</span>)
    - move cursor on any function or module name, and press **shift+tab**: this will bring up the function signitures (can press the "+" sign to expand).
    - **tab**: auto-complete
    - select a block and **tab/shift-tab**: indent/de-indent

In [ ]:
logger.info('loaded widgets')